<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/T5_Translation%20-%20CUDA%20out%20of%20memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece] datasets
!pip install transformers[torch]
!pip install sacrebleu sentencepiece

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("aslg_pc12")
checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    source = example["gloss"]
    target = example["text"]

    # Tokenizing source and target without returning tensors
    tokenized_source = tokenizer(source, truncation=True, padding='max_length', max_length=512)
    tokenized_target = tokenizer(target, truncation=True, padding='max_length', max_length=512)

    return {
        "input_ids": tokenized_source["input_ids"],
        "attention_mask": tokenized_source["attention_mask"],
        "labels": tokenized_target["input_ids"]
    }

#def tokenize_function(example):
#  return tokenizer(example["gloss"], example["text"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.1)
tokenized_datasets["validation"] = tokenized_datasets.pop("test")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Map:   0%|          | 0/87710 [00:00<?, ? examples/s]

In [3]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['gloss', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 78939
    })
    validation: Dataset({
        features: ['gloss', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8771
    })
})

In [4]:
from transformers import TrainingArguments
from transformers import T5ForConditionalGeneration

training_args = TrainingArguments("test-trainer")
model = T5ForConditionalGeneration.from_pretrained(checkpoint, num_labels=2)

In [5]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [6]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: ignored